In [1]:
%load_ext autoreload
%autoreload 2

import pyreadr
import json
import pandas as pd
from models import WRUPredictor

In [2]:
try:
    result = pyreadr.read_r("data/nc_voter_reg_cleanedwbisg.rds")
except FileNotFoundError:
        print(f"Error: File not found.")
        exit()

if not result:
    print("Error: No data found in the .rds file.")
    exit()

df = result[list(result.keys())[0]]
df.head()

,county_desc,zcta5,county_id,status_cd,voter_status_desc,surname,race_code,race_desc,ethnic_desc,party_desc,...,state_name,county_fips,county,state,herf_weight,max_race_prob,plural_race,white_plural,black_plural,other_plural
rownames,,,,,,,,,,,,,,,,,,,,,
1,ALAMANCE,27215,1,A,ACTIVE,BRYAN,W,WHITE,NOT HISPANIC or NOT LATINO,REPUBLICAN,...,North Carolina,37001,001,NC,0.750726,0.861850,white,1.0,0.0,0.0
2,ALAMANCE,27258,1,A,ACTIVE,POPE,W,WHITE,NOT HISPANIC or NOT LATINO,DEMOCRATIC,...,North Carolina,37001,001,NC,0.653723,0.791892,white,1.0,0.0,0.0
3,ALAMANCE,27217,1,A,ACTIVE,WHITESELL,W,WHITE,NOT HISPANIC or NOT LATINO,REPUBLICAN,...,North Carolina,37001,001,NC,0.845644,0.918256,white,1.0,0.0,0.0
4,ALAMANCE,27215,1,A,ACTIVE,KINNEY,W,WHITE,NOT HISPANIC or NOT LATINO,DEMOCRATIC,...,North Carolina,37001,001,NC,0.803672,0.894315,white,1.0,0.0,0.0
5,ALAMANCE,27215,1,A,ACTIVE,MOORE,W,WHITE,UNDESIGNATED,DEMOCRATIC,...,North Carolina,37001,001,NC,0.604750,0.751114,white,1.0,0.0,0.0


In [3]:
key = json.load(open('secrets/CENSUS_API_KEY.json'))
wru_predictor = WRUPredictor(census_api_key=key['CENSUS_API_KEY'])

In [4]:
df = df[['county_desc', 'zcta5', 'county_id', 'surname', 'race_code', 'race_desc', 'state', 'county']]

In [5]:
bisg_result = wru_predictor.inference(df, model='BISG', state="NC", year="2020", census_geo="zcta", census_surname=True, 
                                 surname_only=False, names_to_use="surname", impute_missing=False, skip_bad_geos=True, use_counties=False)
bisg_result

R[write to console]: Collecting 2020 Census data...



Getting census data...


R[write to console]: Predicting race for 2020

R[write to console]: Proceeding with last name predictions...



Predicting race...
ℹ Downloading "wru-data-census_last_c.rds"...
  |======================================================================| 100%
ℹ Downloading "wru-data-first_c.rds"...
  |======================================================================| 100%
ℹ Downloading "wru-data-last_c.rds"...
  |======================================================================| 100%
ℹ Downloading "wru-data-mid_c.rds"...
  |======================================================================| 100%


R[write to console]: Proceeding with Census geographic data at zcta level...

R[write to console]: Using Census geographic data from provided census.data object...

R[write to console]: State 1 of 1: NC

R[write to console]: The following locations in the voter.file are not available in the census data.(listed as state-zcta):
NC-23917, NC-24148, NC-24333, NC-24348, NC-24506, NC-27010, NC-27049, NC-27116, NC-27228, NC-27552, NC-27699, NC-27802, NC-27811, NC-27835, NC-27836, NC-27838, NC-27854, NC-27859, NC-27867, NC-27868, NC-27887, NC-28010, NC-28035, NC-28038, NC-28123, NC-28145, NC-28446, NC-28522, NC-28583, NC-28608, NC-28647, NC-28661, NC-28687, NC-28699, NC-28703, NC-28728, NC-28737, NC-28750, NC-28765, NC-28776

R[write to console]: NOTE: Skipping unavailable geolocations. Returning partial data set.



ℹ All local files already up-to-date!


R[write to console]: 328614 (4.9%) individuals' last names were not matched.



,county_desc,zcta5,county_id,surname,race_code,race_desc,state,county,white,black,api,native,multiple,hispanic
rownames,,,,,,,,,,,,,,
1,ALAMANCE,27215,1,BRYAN,W,WHITE,NC,001,0.927419,0.005411,0.003064,0.0,0.0,0.040541
2,ALAMANCE,27258,1,POPE,W,WHITE,NC,001,0.960416,0.001932,0.000521,0.0,0.0,0.020214
3,ALAMANCE,27217,1,WHITESELL,W,WHITE,NC,001,0.943166,0.004035,0.003370,0.0,0.0,0.026183
4,ALAMANCE,27215,1,KINNEY,W,WHITE,NC,001,0.927419,0.005411,0.003064,0.0,0.0,0.040541
5,ALAMANCE,27215,1,MOORE,W,WHITE,NC,001,0.969125,0.001013,0.002220,0.0,0.0,0.013386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25843100,WATAUGA,28608,95,MARTIN,B,BLACK or AFRICAN AMERICAN,NC,189,0.528471,0.187450,0.082620,0.0,0.0,0.152203
25844100,YANCEY,28755,100,HOILMAN,W,WHITE,NC,199,0.275306,0.545091,0.023248,0.0,0.0,0.110785
25845100,YANCEY,28755,100,LINDENAUX,W,WHITE,NC,199,0.709369,0.132725,0.031860,0.0,0.0,0.075789


In [ ]:
# fbisg_result = wru_predictor.inference(df, model='fBISG', state="NC", year="2020", census_geo="zcta", census_surname=True, 
#                                        surname_only=False, names_to_use="surname", impute_missing=True, skip_bad_geos=True, use_counties=False)
# fbisg_result

R[write to console]: Predicting race for 2020

R[write to console]: Using `predict_race` to obtain initial race prediction priors with BISG model

R[write to console]: Predicting race for 2020

R[write to console]: Proceeding with last name predictions...



Getting census data...
Predicting race...
ℹ All local files already up-to-date!


R[write to console]: Proceeding with Census geographic data at zcta level...

R[write to console]: Using Census geographic data from provided census.data object...

R[write to console]: State 1 of 1: NC

R[write to console]: The following locations in the voter.file are not available in the census data.(listed as state-zcta):
NC-23917, NC-24148, NC-24333, NC-24348, NC-24506, NC-27010, NC-27049, NC-27116, NC-27228, NC-27552, NC-27699, NC-27802, NC-27811, NC-27835, NC-27836, NC-27838, NC-27854, NC-27859, NC-27867, NC-27868, NC-27887, NC-28010, NC-28035, NC-28038, NC-28123, NC-28145, NC-28446, NC-28522, NC-28583, NC-28608, NC-28647, NC-28661, NC-28687, NC-28699, NC-28703, NC-28728, NC-28737, NC-28750, NC-28765, NC-28776

R[write to console]: NOTE: Skipping unavailable geolocations. Returning partial data set.



ℹ All local files already up-to-date!


R[write to console]: 328614 (4.9%) individuals' last names were not matched.



ℹ All local files already up-to-date!


R[write to console]: fBISG relies on MCMC; for reproducibility, I am setting RNG seed and returning it as attribute 'RNGseed'.
To silence this message, you can set a seed explicitly by defining the 'seed' element in the control list.

R[write to console]: Forming Pr(race | location) tables from census data...


R[write to console]: Error in predict_race_me(voter.file = voter.file, names.to.use = names.to.use,  : 
  Some records in voter.file have geographic locations that I wasn't able to find in the census.data.

          Records may have mis-matched geographic units that do not exist in the census.



RRuntimeError: Error in predict_race_me(voter.file = voter.file, names.to.use = names.to.use,  : 
  Some records in voter.file have geographic locations that I wasn't able to find in the census.data.

          Records may have mis-matched geographic units that do not exist in the census.


In [ ]:
# bifsg_result = wru_predictor.inference(df, model='BISG', state="NC", year="2020", census_geo="zcta", census_surname=True, surname_only=False, 
#                                        names_to_use="surname, first", impute_missing=False, skip_bad_geos=True, use_counties=False)
